<a href="https://colab.research.google.com/github/VeereshShringari/NLP-DHS-SRK/blob/master/7_WordEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Custom Word Embeddings

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_path = "/content/drive/My Drive/DHS2019_Workshop/Disaster/"
# Please enter the file name
file_name = data_path + "socialmedia_disaster_tweets.csv"
# Please enter the name of the text column
col_name = "text"

### Installing & Importing H2O

In [0]:
!pip install h2o

     |████████████████████████████████| 123.7MB 37kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.10-py2.py3-none-any.whl size=123719583 sha256=fce319e47bf16d39460a45784d99c1f041d2629c2f1c03aac9d509ed6913b5e3
  Stored in directory: /root/.cache/pip/wheels/7d/09/bc/23eaa75a5948deb1d6c88d9ee3a20f94bac7d9df6c55ec8aa3
Successfully built h2o


In [0]:
import h2o
h2o.init()
from h2o.estimators.word2vec import H2OWord2vecEstimator

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp2275fl4r
  JVM stdout: /tmp/tmp2275fl4r/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp2275fl4r/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,"7 days, 16 hours and 37 minutes"
H2O cluster name:,H2O_from_python_unknownUser_yi088r
H2O cluster total nodes:,1
H2O cluster free memory:,3 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


### Data Preparation

In [0]:
df = h2o.import_file(file_name, header=1, sep=",")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
df = df[[col_name]].ascharacter()

In [0]:
df

text
Just happened a terrible car crash
Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
"Heard about #earthquake is different cities, stay safe everyone."
"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
Forest fire near La Ronge Sask. Canada
All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
"13,000 people receive #wildfires evacuation orders in California"
Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school
#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
Apocalypse lighting. #Spokane #wildfires


### Tokenization

In [0]:
def tokenize(sentences):
    # tokenize the sentences
    tokenized = sentences.tokenize("\\W+")
    # lower case the text column
    tokenized = tokenized.tolower()
    # filter out the sentences which has less than 2 characters or where text is missing
    tokenized = tokenized[(tokenized.nchar() >= 2) | (tokenized.isna()),:]
    return tokenized

words = tokenize(df[col_name])

### Word2Vec Model

The next step is to build the word2vec model. We can also adjust the parameters of the word2vec mdoel. Please refer to the [documentation of H2oWord2vecEstimator](http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/modeling.html#h2oword2vecestimator) for more details on the parameters.

In [0]:
print("Build word2vec model")
w2v_model = H2OWord2vecEstimator(min_word_freq=3,
                                 vec_size=300,
                                 window_size=5,
                                 epochs=10,
                                 word_model="skip_gram")
w2v_model.train(training_frame=words)

Build word2vec model
word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [0]:
w2v_model.to_frame().as_data_frame().to_csv(data_path + "w2vec.txt", float_format='%.6f', sep=" ", header=False, index=False)

In [0]:
w2v_model.to_frame().as_data_frame().head()

,Word,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,...,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300
0,raw,0.079444,0.093110,0.034768,0.010234,0.074323,0.032686,-0.104307,0.009156,0.163026,-0.083421,-0.008091,-0.013459,-0.081597,0.022943,0.112029,-0.086098,0.000531,0.062078,-0.023701,0.072399,0.058407,0.165790,-0.033303,0.034051,-0.014061,-0.252406,0.024421,-0.236130,0.054420,-0.078156,-0.101891,-0.009711,-0.123936,0.075986,0.111765,-0.026073,0.073905,0.087416,-0.131131,...,-0.113861,0.027511,0.057393,-0.064332,-0.184280,-0.048194,-0.071549,-0.120024,-0.193569,0.114609,-0.059336,0.086683,0.072594,0.040357,-0.020176,-0.114048,-0.117249,-0.091184,0.135161,0.075102,0.009249,-0.137748,-0.023941,0.015440,-0.156949,0.005079,0.106412,0.084194,-0.001251,0.124868,0.049862,0.102021,0.008159,-0.163186,-0.287354,0.158501,0.048454,0.069939,0.138541,0.090188
1,ye,0.131668,0.041584,-0.025391,-0.017545,0.060882,-0.006810,-0.012733,-0.024944,0.100231,-0.038912,-0.143685,-0.002235,-0.044242,0.037241,-0.121083,0.004897,-0.060071,-0.060219,0.138772,-0.011019,0.077987,0.120597,-0.170680,0.103392,-0.045546,0.098658,-0.080290,-0.169892,0.047348,0.004033,-0.123169,0.056839,0.058239,-0.073453,0.221603,0.102738,-0.067367,-0.056070,-0.025048,...,-0.057124,0.025673,-0.047504,0.012523,-0.023397,-0.146573,-0.112593,-0.051972,-0.015044,0.037370,-0.168326,0.028413,-0.153810,-0.040679,0.018243,-0.197635,0.043771,0.119580,0.026226,0.072028,-0.075010,0.064485,0.023471,-0.078052,-0.093717,0.189189,0.222901,0.189139,0.158197,0.210296,-0.146494,-0.054572,0.077473,0.102514,0.018405,0.017132,0.163586,0.056901,0.041456,0.136590
2,therefore,0.009920,-0.057150,-0.043196,-0.049632,-0.018891,0.061795,-0.028184,-0.206557,0.138644,-0.074551,0.102299,-0.101645,-0.117659,0.014261,-0.122883,0.007799,0.116477,-0.136648,-0.102615,-0.060173,0.065940,0.019577,0.073943,0.021793,0.004111,-0.154087,-0.044217,0.027580,-0.074014,0.052530,-0.007895,0.151458,-0.104271,-0.032785,0.032494,0.072682,0.000726,-0.071978,-0.097731,...,-0.118544,0.011496,-0.036489,0.145100,0.065969,-0.100366,-0.178530,-0.076942,0.022023,-0.091009,-0.001274,0.023554,-0.124641,-0.037965,-0.073606,-0.051614,0.158334,0.090684,-0.044498,0.185193,-0.054364,-0.000364,-0.033463,-0.045963,-0.036176,-0.000563,0.056981,0.067183,0.222455,-0.004403,-0.068266,0.049223,0.182952,0.023493,-0.065389,0.109215,0.070019,0.058927,-0.013969,0.007419
3,golan,-0.031720,-0.279020,-0.129370,-0.131776,-0.098221,-0.011754,-0.074456,-0.031258,0.253173,0.009621,0.005261,-0.076342,-0.036331,0.160958,0.022501,-0.010773,-0.176208,0.047474,0.099284,-0.113808,-0.055190,0.067126,0.286171,0.099151,0.087439,-0.201981,-0.046615,-0.163119,0.083000,0.113521,-0.147437,0.056449,-0.059407,-0.022723,-0.104497,0.001438,-0.126024,-0.008641,-0.212521,...,-0.364419,-0.009586,-0.148458,0.242190,-0.153027,0.116846,-0.175072,-0.266199,-0.102560,-0.127115,-0.085944,-0.120678,0.106273,-0.137091,-0.315652,-0.006452,-0.292274,0.090369,0.211562,0.202805,0.076336,0.067323,-0.390901,0.228834,-0.151191,0.409750,-0.070225,-0.137520,-0.117254,0.027920,-0.084532,0.060691,0.074852,0.015463,-0.097874,0.266472,0.019585,-0.016301,-0.112680,0.082549
4,jailed,0.025730,0.070001,0.038100,-0.097845,0.016363,0.042186,0.001034,0.021435,0.063607,-0.054940,-0.002456,-0.091080,-0.063421,0.011765,-0.001709,0.030810,-0.052844,-0.093112,-0.002267,0.024529,-0.050082,0.032726,-0.101315,0.007227,0.043278,0.001448,0.104812,-0.107141,0.043511,-0.105729,-0.036581,0.038271,-0.035834,-0.083160,0.256695,-0.023400,0.081828,-0.141057,-0.080424,...,-0.065390,0.053610,0.000163,-0.026734,-0.103723,-0.066825,-0.018208,-0.046789,-0.053959,0.054094,-0.126286,0.088566,0.000916,0.173437,0.037724,-0.147377,-0.011931,0.022628,0.100234,0.045227,-0.079044,0.0333